<a href="https://colab.research.google.com/github/JohnRoeder5/Stat_arbitrage_analysis/blob/main/stat_arbitrage_Jroeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

starting tickers: VMI, RS, FLS, DHR

steps:
1. find the correlation between stocks. need to be in similar markets and need to be cointegrated
2. determine stationarity
3. determine threshold (z-score)
4. consider using moving averages and mean reversion.


In [ ]:
#imports
import threading
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sea
import yfinance as yf
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint

from sklearn import svm
from datetime import datetime, timedelta
import math
import requests
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics



REMEMBER THAT .tail() gets the most recent closing prices and .head() gets the oldest.


In [ ]:
#list of tickers. Get tickrs from yfinance


#tickers = ['CCI', 'AMT', 'SBAC']
tickers = ['WFC', 'BAC', 'JPM', 'MS', 'GS', 'C', 'USB']
#tickers = ['DUK', 'NEE', 'AEP']
#tickers = ['VMI', 'RS', 'FLS', 'DHR', 'CCI', 'AMT', 'SBAC', 'DUK', 'NEE', 'AEP', 'WFC', 'BAC', 'JPM', 'MS', 'GS', 'C', 'USB']
#tickers = ['ETH-USD', 'BTC-USD']
#tickers = ['JPM', 'BAC', 'WFC', 'C', 'GS', 'MS', 'DB', 'BBVA', 'SAN', 'ING', 'UBS', 'HSBC', 'PNC', 'USB', 'BK', 'STT', 'KEY', 'RF', 'HBAN', 'FITB', 'BLK', 'MTB', 'ZION', 'FFIN', 'FHN', 'UBSI','SBCF', 'TCBI', 'BOKF', 'PFG', 'GBCI', 'TFC', 'CFR', 'UMBF', 'FULT', 'ONB', 'INDB', 'IBOC']
print(len(tickers))
def clean_dataset(dataset):
    # Interpolate missing values using linear interpolation
    dataset.interpolate(method='linear', axis=0, inplace=True)
    dataset.ffill(inplace=True)
    dataset.bfill(inplace=True)

    # Check if there are still NaN, None, or Inf values after interpolation
    if dataset.isnull().values.any() or (dataset == np.inf).any().any() or (dataset == -np.inf).any().any():
        return None  # If there are, return None

    # Replace inf and -inf with finite numbers (you can adjust this based on your needs)
    dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Replace NaN after handling inf
    dataset.fillna(0, inplace=True)

    # Replace None with a specific value, e.g., 0
    dataset.replace([None], 0, inplace=True)

    return dataset

dfs_dict = {}

# Create individual dfs
for ticker in tickers:
    get_data = yf.download(ticker, start='2013-01-01')
    cleaned_data = clean_dataset(get_data)

    if cleaned_data is not None:
        dfs_dict[ticker] = cleaned_data

print(len(dfs_dict))




get the df of the closing prices for the stocks. plot the correlation coefficient heat map


In [ ]:
#WHEN U GET NEW STOCKS NEED OT REPLACE THE TICKERS IN THE DICTIONARY

#all_stocks_close = pd.concat([dfs_dict['VMI']['Close'], dfs_dict['RS']['Close'], dfs_dict['FLS']['Close'], dfs_dict['DHR']['Close']], axis=1)
#all_stocks_close = pd.concat([dfs_dict['CCI']['Close'], dfs_dict['AMT']['Close'], dfs_dict['SBAC']['Close']], axis=1)
#all_stocks_close = pd.concat([dfs_dict['DUK']['Close'], dfs_dict['NEE']['Close'], dfs_dict['AEP']['Close']], axis=1)
#all_stocks_close = pd.concat([dfs_dict['WFC']['Close'], dfs_dict['BAC']['Close'], dfs_dict['JPM']['Close']], axis=1)
# all_stocks_close = pd.concat([dfs_dict['ETH-USD']['Close'], dfs_dict['BTC-USD']['Close']], axis=1)

all_stocks_close = pd.concat([dfs_dict[ticker]['Close'] for ticker in tickers], axis=1, keys=tickers)
print(all_stocks_close.shape)
#check if the column is NaN fir last price then replace with yesterdays close if it is
# print(all_stocks_close.tail())
for column in all_stocks_close.columns:
  last_price = all_stocks_close[column].iloc[-1]
  second_to_last_price= all_stocks_close[column].iloc[-2]
  if(np.isnan(last_price)):
    all_stocks_close[column].iloc[-1] = second_to_last_price

# print(all_stocks_close.head())

new_column_names = [f'{ticker}_Close' for ticker in tickers]
print(new_column_names)
# Rename the columns
all_stocks_close.columns = new_column_names


#check correlations using ehatmap:
fig, ax = plt.subplots(figsize=(25, 20))
sea.heatmap(all_stocks_close.pct_change().corr(method ='pearson'), ax=ax, cmap='coolwarm', annot=True, fmt=".2f")
ax.set_title('Assets Correlation Matrix')



In [ ]:
#finding cointegrated pairs:
def get_coint_pairs(data):
    n = data.shape[1]
    print(n)
    p_matrix = np.ones((n, n))
    keys = data.keys()
    print(keys)
    pairs = []
    for i in range(n):
        for j in range(i+1, n):
          try:
            result = coint(data[keys[i]], data[keys[j]])
            p_matrix[i, j] = result[1]
            if result[1] < 0.05:
                pairs.append((keys[i], keys[j]))
          except Exception as e:
            print(f"Error in key pair: {keys[i]}, {keys[j]}")
            print(e)
            return None






            # result = coint(data[keys[i]], data[keys[j]])
            # p_matrix[i, j] = result[1]
            # if result[1] < 0.05:
            #   pairs.append((keys[i], keys[j]))
    return p_matrix, pairs
# calculate p-values and plot as a heatmap
pvalues, pairs = get_coint_pairs(all_stocks_close)

for pair in pairs:
  print(pair)
fig, ax = plt.subplots(figsize=(25, 20))
sea.heatmap(pvalues, xticklabels = all_stocks_close.columns,
                yticklabels = all_stocks_close.columns, cmap = 'RdYlGn_r', annot = True, fmt=".2f",
                mask = (pvalues >= 0.05))
ax.set_title('Assets Cointregration Matrix p-values Between Pairs')
plt.tight_layout()

In [ ]:
#test for futhr stationaruyt using the Augemnted dcikey fuller

best_value = []
for pair in pairs:
  asset1, asset2 = pair
  print(pair)
  train = pd.DataFrame()
  train['asset1'] = all_stocks_close[asset1]
  train['asset2'] = all_stocks_close[asset2]
  #visualize the prces
  # ax = train[['asset1','asset2']].plot(figsize=(12, 6), title = 'Daily Closing Prices for {} and {}'.format(asset1,asset2))
  # ax.set_ylabel("Closing Price")
  # ax.grid(True);

  # OLS regresion
  model=sm.OLS(train.asset2, train.asset1).fit()


  # print(f'Regression Summary for {asset1} and {asset2}:\n{model.summary()}')



  print(f'Hedge Ratio = {model.params[0]}')
  # results
  # plt.rc('figure', figsize=(12, 7))
  # plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 16}, fontproperties = 'monospace')
  # plt.axis('off')
  # plt.subplots_adjust(left=0.2, right=0.8, top=0.7, bottom=0.1)



  spread = train.asset2 - model.params[0] * train.asset1

  z_score = (spread - spread.mean()) / np.std(spread)


  z_upper_limit = np.mean(z_score) + np.std(z_score)
  z_lower_limit = np.mean(z_score) - np.std(z_score)


  plt.figure(figsize=(12, 6))
  ax = spread.plot(title="Pair's Spread")
  ax.set_ylabel("Spread")
  ax.grid(True)
  ax.legend(labels=[asset1+ ' '+asset2+ " Spread"])

  ax2 = plt.subplot(2, 1, 2)
  z_score.plot(ax=ax2, color='orange', label='Z-Score')
  ax2.axhline(y=z_upper_limit, color='red', linestyle='--', label='Z Upper Limit')
  ax2.axhline(y=z_lower_limit, color='green', linestyle='--', label='Z Lower Limit')
  ax2.set_ylabel("Z-Score")
  ax2.set_xlabel("Date")
  ax2.legend()



  # ADF test
  adf = adfuller(spread, maxlag = 1)
  print(f'ADF Test for {asset1} and {asset2}:')
  print(f'Critical Value = {adf[0]}')
  # criticl values
  print(adf)


  adf_dict = adf[4]

  print(adf_dict['1%'])
  plt.tight_layout()
  plt.show()
  print('\n')
  print('\n')
  #if critical val less than the smallest confidenece intervla add to list to use for strat.
  if(adf[0] < adf_dict['5%']):
    best_value.append(pair)

print(len(best_value))

In [ ]:
def calculate_sharpe_ratio(returns, risk_free_rate):
    mean_return = returns.mean()
    risk = returns.std()
    sharpe_ratio = (mean_return - risk_free_rate) / risk
    return sharpe_ratio

def plot_strategy(signals):
    # initial capital to calculate the actual pnl
    initial_capital = 100000

    # shares to buy for each position
    positions1 = initial_capital// max(signals['asset1'])
    positions2 = initial_capital// max(signals['asset2'])

    # since there are two assets, we calculate each asset Pnl
    # separately and in the end we aggregate them into one portfolio
    portfolio = pd.DataFrame()
    portfolio['asset1'] = signals['asset1']
    portfolio['holdings1'] = signals['positions1'].cumsum() * signals['asset1'] * positions1
    portfolio['cash1'] = initial_capital - (signals['positions1'] * signals['asset1'] * positions1).cumsum()
    portfolio['total asset1'] = portfolio['holdings1'] + portfolio['cash1']
    portfolio['return1'] = portfolio['total asset1'].pct_change()
    portfolio['positions1'] = signals['positions1']
    # pnl for the 2nd asset
    portfolio['asset2'] = signals['asset2']
    portfolio['holdings2'] = signals['positions2'].cumsum() * signals['asset2'] * positions2
    portfolio['cash2'] = initial_capital - (signals['positions2'] * signals['asset2'] * positions2).cumsum()
    portfolio['total asset2'] = portfolio['holdings2'] + portfolio['cash2']
    portfolio['return2'] = portfolio['total asset2'].pct_change()
    portfolio['positions2'] = signals['positions2']

    # total pnl and z-score
    portfolio['z'] = signals['z']
    portfolio['total asset'] = portfolio['total asset1'] + portfolio['total asset2']
    portfolio['z upper limit'] = signals['z upper limit']
    portfolio['z lower limit'] = signals['z lower limit']
    portfolio = portfolio.dropna()

    risk_free_rate = 0.02

    portfolio['total_return'] = portfolio['total asset'].pct_change()
    sharpe_ratio_total = calculate_sharpe_ratio(portfolio['total_return'], risk_free_rate)

    #sharpe ratio calc
    print("Sharpe Ratio for Total Portfolio:", sharpe_ratio_total)

    # plot the asset value change of the portfolio and pnl along with z-score
    fig = plt.figure(figsize=(14,6),)
    ax = fig.add_subplot(111)
    ax2 = ax.twinx()
    l1, = ax.plot(portfolio['total asset'], c='g')
    l2, = ax2.plot(portfolio['z'], c='black', alpha=0.3)
    b = ax2.fill_between(portfolio.index,portfolio['z upper limit'],portfolio['z lower limit'], alpha=0.2,color='#ffb48f')
    ax.set_ylabel('Asset Value')
    ax2.set_ylabel('Z Statistics',rotation=270)
    ax.yaxis.labelpad=15
    ax2.yaxis.labelpad=15
    ax.set_xlabel('Date')
    ax.xaxis.labelpad=15
    plt.title('Portfolio Performance with Profit and Loss')
    plt.legend([l2,b,l1],['Z Statistics','Z Statistics +-1 Sigma','Total Portfolio Value'],loc='upper left');


    # calculate CAGR
    final_portfolio = portfolio['total asset'].iloc[-1]
    delta = (portfolio.index[-1] - portfolio.index[0]).days
    print('Number of days = ', delta)
    YEAR_DAYS = 365
    returns = (final_portfolio/initial_capital) ** (YEAR_DAYS/delta) - 1
    print('CAGR = {:.3f}%' .format(returns * 100))
    print(portfolio.tail())

In [ ]:
# calculate z-score
def zscore(series):
 return (series - series.mean()) / np.std(series)
# create a dataframe for trading signals

#receives the list of the best pairs calcuated int he last section.
def plot_relationships(best_value, asset1, asset2):

    # visualize trading signals and position
    fig=plt.figure(figsize=(14,6))
    bx = fig.add_subplot(111)
    bx2 = bx.twinx()


    #plot two different assets
    l1, = bx.plot(signals['asset1'], c='#4abdac', label= "asset 1")
    l2, = bx2.plot(signals['asset2'], c='#907163', label = "asset 2")

    bx.plot(signals['65_MA_asset1'], label='Asset 1 65-MA', linestyle='--', color='green')
    bx2.plot(signals['65_MA_asset2'], label='Asset 2 65-MA', linestyle='--', color='blue')

    bx.plot(signals['200_MA_asset1'], label='Asset 1 200-MA', linestyle='--', color='red')
    bx2.plot(signals['200_MA_asset2'], label='Asset 2 200-MA', linestyle='--', color='yellow')

    #To have all signals uncomment this:
    u1, = bx.plot(signals['asset1'][signals['positions1'] == 1], lw=0, marker='^', markersize=8, c='green',alpha=0.7, label = 'Buy Signal Asset 1')
    d1, = bx.plot(signals['asset1'][signals['signals1'] == -1], lw=0, marker='v', markersize=8, c='red', alpha=0.7, label='Sell Signal Asset 1')
    u2, = bx2.plot(signals['asset2'][signals['signals2'] == 1], lw=0, marker='^', markersize=8, c='blue', alpha=0.7, label='Buy Signal Asset 2')
    d2, = bx2.plot(signals['asset2'][signals['signals2'] == -1], lw=0, marker='v', markersize=8, c='orange', alpha=0.7, label='Sell Signal Asset 2')

    # # plot when there are opposing signals
    # u1, = bx.plot(opposing_signals['asset1'][signals['positions1'] == 1], lw=0, marker='^', markersize=8, c='green', alpha=0.7, label='Buy Signal Asset 1')
    # d1, = bx.plot(opposing_signals['asset1'][signals['positions1'] == -1], lw=0, marker='v', markersize=8, c='red', alpha=0.7, label='Sell Signal Asset 1')
    # u2, = bx2.plot(opposing_signals['asset2'][signals['positions2'] == 1], lw=0, marker='^', markersize=8, c='blue', alpha=0.7, label='Buy Signal Asset 2')
    # d2, = bx2.plot(opposing_signals['asset2'][signals['positions2'] == -1], lw=0, marker='v', markersize=8, c='orange', alpha=0.7, label='Sell Signal Asset 2')

    bx.axhline(y=signals['z upper limit'][0], color='gray', linestyle='--', label='Z Upper Limit')
    bx.axhline(y=signals['z lower limit'][0], color='gray', linestyle='--', label='Z Lower Limit')



    bx.set_ylabel('Asset 1 Price', color='#4abdac')
    bx2.set_ylabel('Asset 2 Price', color='#907163')
    bx.set_xlabel('Date')
    bx.set_title('Asset Prices + Trading Signals')

    #legends
    bx.legend(handles=[l1, u1, d1], labels=[ asset1 +  ' Price', 'Buy Signal '+asset1, 'Sell Signal '+asset1], loc='upper left')
    bx2.legend(handles=[l2, u2, d2], labels=[asset2+ ' Price', 'Buy Signal '+asset2, 'Sell Signal '+asset2], loc='upper right')

    # Show the plot
    plt.tight_layout()
    plt.show()

for pair in best_value:
    asset1, asset2 = pair
    print(pair[0])
    print(pair[1])
    signals = pd.DataFrame()
    signals['asset1'] = all_stocks_close[asset1]
    signals['asset2'] = all_stocks_close[asset2]
    ratios = signals.asset1 / signals.asset2
    #print(signals)


    signals['65_MA_asset1'] = signals['asset1'].rolling(window=65).mean()
    signals['200_MA_asset1'] = signals['asset1'].rolling(window=200).mean()

    signals['65_MA_asset2'] = signals['asset2'].rolling(window=65).mean()
    signals['200_MA_asset2'] = signals['asset2'].rolling(window=200).mean()

    signals['ma_signal1'] = 0
    signals.loc[signals['65_MA_asset1'] > signals['200_MA_asset1'], 'ma_singal1']  = 1  # Buy signal
    signals.loc[signals['65_MA_asset1'] < signals['200_MA_asset1'], 'ma_singal1'] = -1  # Sell signal

    # generate trading signals for asset2
    signals['ma_signal2'] = 0
    signals.loc[signals['65_MA_asset2'] > signals['200_MA_asset2'], 'ma_singal2'] = 1  # Buy signal
    signals.loc[signals['65_MA_asset2'] < signals['200_MA_asset2'], 'ma_singal2']= -1  # Sell signal




    #get z-score and find the thresholds
    signals['z'] = zscore(ratios)
    signals['z upper limit'] = np.mean(signals['z']) + np.std(signals['z'])
    signals['z lower limit'] = np.mean(signals['z']) - np.std(signals['z'])


    # create signal - short if z-score is greater than upper limit else long
    signals['signals1'] = 0
    signals['signals1'] = np.select([signals['z'] > signals['z upper limit'], signals['z'] < signals['z lower limit']], [-1, 1], default=0)


    # we take the first order difference to obtain portfolio position in that stock
    signals['positions1'] = signals['signals1'].diff()
    signals['signals2'] = -signals['signals1']
    signals['positions2'] = signals['signals2'].diff()

    signals['combined_signal1'] = signals['ma_signal1'] + signals['signals1']
    signals['combined_signal2'] = signals['ma_signal2'] + signals['signals2']




    # verify datafame head and tail
    signals.head(3).append(signals.tail(3))
    opposing_signals = signals[(signals['signals1'] != 0) & (signals['signals1'] == -signals['signals2'])]

    plot_strategy(signals)

    plot_relationships(best_value, asset1, asset2)

